Attempting to clean some of the data for the Nashville E911 project.
Notebook by Chris Mulvey as part of a project at Nashville Software School.
16 Novenber 2020.

In [3]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [4]:
derecho_calls = pd.read_csv('../data/911_Phone Calls_Derecho_050320 050420.csv')
derecho_computer = pd.read_csv('../data/Computer Aided Dispatch Data_Derecho Incidents 050320-050420.csv',
                               skiprows = 3)
tornado_calls = pd.read_csv('../data/911_Phone_Calls_Tornado_030320.csv')
tornado_computer = pd.read_csv('../data/Computer Aided Dispatch Data_Tornado Incidents 030320.csv', skiprows = 3)

In [5]:
# Convert Seizure DateTime to a timestamp
tornado_calls['Timestamp'] = pd.to_datetime(tornado_calls['Seizure DateTime'])

In [6]:
# Subset tornado_calls to capture only calls after 2 am
late_tornado_calls = tornado_calls[tornado_calls['Timestamp'] >= '2020-03-03 02:00:00']

In [7]:
late_tornado_calls.head()

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address,Timestamp
292,3/3/2020 2:00,911 Calls,36.154633,-86.784500,PRELIM 1033 DEMONBREUN ST - SE Sector,2020-03-03 02:00:00
293,3/3/2020 2:00,911 Calls,36.182045,-86.749270,WIRELESS CALLER,2020-03-03 02:00:00
294,3/3/2020 2:00,911 Calls,36.176723,-86.812613,2101-D HEIMAN ST - SECTOR N,2020-03-03 02:00:00
295,3/3/2020 2:00,911 Calls,NaN,NaN,310 N 9TH ST,2020-03-03 02:00:00
296,3/3/2020 2:01,911 Calls,36.183674,-86.624687,4914 OLD HICKORY BLVD - S Sector,2020-03-03 02:01:00


In [8]:
late_tornado_calls.value_counts('Cell Tower Address')

Cell Tower Address
WIRELESS CALLER                        54
1051G CHICAMAUGA AVE - S                6
4914 OLD HICKORY BLVD - S Sector        4
4884 LICKTON PIKE - SE                  4
1608F ROSA L PARKS BLVD - S             3
                                       ..
5060 TIDWELL HOLLOW RD - SW             1
508 INTERSTATE BLVD S - SW              1
5171 HICKORY HOLLOW PKY - SE            1
5171 HICKORY HOLLOW PKY - SE Sector     1
1 TERMINAL DR                           1
Length: 102, dtype: int64

There doesn't seem to be a good wway to clean up the wireless caller value.

In [9]:
# Convert Seizure DateTime to a timestamp
tornado_computer['Timestamp'] = pd.to_datetime(tornado_computer['IncidentDate'])

In [12]:
# Subset tornado_calls to capture only calls after 2 am
late_tornado_comp = tornado_computer[tornado_computer['Timestamp'] >= '2020-03-03 02:00:00']

In [13]:
late_tornado_comp.head()

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1,Timestamp
129,3/3/2020 2:01:33 AM,1421 WOODLAND ST,36.177240,-86.743885,SMOKE DETECTOR ALARM HOUSE,2020-03-03 02:01:33
130,3/3/2020 2:01:52 AM,2658 BARCLAY DR,36.174243,-86.719824,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,2020-03-03 02:01:52
131,3/3/2020 2:02:32 AM,S 9TH ST / MAIN ST,36.175964,-86.757246,OUTSIDE NATURAL GAS ODOR COMMERCIAL LINE TANK ...,2020-03-03 02:02:32
132,3/3/2020 2:03:06 AM,111 EVANDER ST,36.175976,-86.740844,OUTSIDE ODOR,2020-03-03 02:03:06
133,3/3/2020 2:04:00 AM,938 W EASTLAND AV,36.184138,-86.754363,OUTSIDE ODOR,2020-03-03 02:04:00


In [14]:
late_tornado_comp.value_counts('IncidentTypeDescription1')

IncidentTypeDescription1
OUTSIDE ODOR                                             17
WIRES DOWN WITH NO SMOKE OR ARCING                        9
GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD               8
ELECTRICAL HAZARD WITH OR NEAR WATER                      7
CITIZEN ASSIST  NON MEDICAL                               6
                                                         ..
ODOR OF SMOKE OUTSIDE                                     1
WIRES DOWN WITH NO SMOKE OR ARCING POWER DISCONNECTED     1
HOUSE NATURAL GAS ODOR                                    1
HOUSE FIRE                                                1
ABDOMINAL PAIN - NOT ALERT                                1
Length: 62, dtype: int64

There may be some opportunity to clean this data. There seem to be several different descriptions that speak about electrical hazards or wires down, written in different ways.

In [17]:
tornado_electrical = late_tornado_comp[late_tornado_comp['IncidentTypeDescription1'].str.contains('ELECTRICAL')]

In [18]:
tornado_electrical

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1,Timestamp
178,3/3/2020 2:27:49 AM,1740 61ST AV N,36.171185,-86.857872,ELECTRICAL ARCING,2020-03-03 02:27:49
206,3/3/2020 3:11:24 AM,422 RAINTREE PL,36.178102,-86.601962,ELECTRICAL HAZARD WITH OR NEAR WATER,2020-03-03 03:11:24
213,3/3/2020 3:22:38 AM,317 E MARTHONA RD,36.260279,-86.727415,ELECTRICAL HAZARD WITH OR NEAR WATER,2020-03-03 03:22:38
237,3/3/2020 4:23:50 AM,ELLINGTON PKWY S / SPRING ST,36.176031,-86.766434,ELECTRICAL INVESTIGATION UNKNOWN SITUATION,2020-03-03 04:23:50
241,3/3/2020 4:30:49 AM,2322 SELMA AV,36.175715,-86.685965,ELECTRICAL HAZARD WITH OR NEAR WATER,2020-03-03 04:30:49
243,3/3/2020 4:38:24 AM,NEWHALL DR / BRITTANY DR,36.176193,-86.723042,ELECTRICAL HAZARD WITH OR NEAR WATER,2020-03-03 04:38:24
245,3/3/2020 4:40:25 AM,1736 24TH AV N,36.178917,-86.815401,ELECTRICAL HAZARD WITH OR NEAR WATER,2020-03-03 04:40:25
251,3/3/2020 4:50:09 AM,LINCOYA DR / MERLYN LN,36.176734,-86.675620,ELECTRICAL HAZARD WITH OR NEAR WATER,2020-03-03 04:50:09
254,3/3/2020 4:51:36 AM,821 RAMSEY ST,36.177137,-86.757858,ELECTRICAL HAZARD WITH OR NEAR WATER,2020-03-03 04:51:36


In [19]:
tornado_electrical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 178 to 254
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   IncidentDate              9 non-null      object        
 1   Location                  9 non-null      object        
 2   Latitude1                 9 non-null      float64       
 3   Longitude1                9 non-null      float64       
 4   IncidentTypeDescription1  9 non-null      object        
 5   Timestamp                 9 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 504.0+ bytes


In [20]:
tornado_wires = late_tornado_comp[late_tornado_comp['IncidentTypeDescription1'].str.contains('WIRE')]

In [22]:
tornado_wires

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1,Timestamp
138,3/3/2020 2:05:21 AM,206 S 10TH ST,36.175023,-86.752005,WIRES DOWN WITH NO SMOKE OR ARCING,2020-03-03 02:05:21
140,3/3/2020 2:05:28 AM,2129 BROOKVIEW DR,36.176634,-86.696201,WIRES DOWN WITH NO SMOKE OR ARCING,2020-03-03 02:05:28
142,3/3/2020 2:05:44 AM,MERIDIAN ST / MERIDIAN CT,36.198063,-86.766851,WIRES DOWN WITH SMOKE OR ARCING,2020-03-03 02:05:44
143,3/3/2020 2:06:38 AM,149 BARKER RD,36.177990,-86.700407,WIRES DOWN WITH NO SMOKE OR ARCING,2020-03-03 02:06:38
157,3/3/2020 2:11:13 AM,MCGAVOCK PKE / STINSON RD,36.172855,-86.679306,WIRES DOWN WITH NO SMOKE OR ARCING,2020-03-03 02:11:13
160,3/3/2020 2:14:34 AM,2602 BRITTANY DR,36.176780,-86.714962,WIRES DOWN WITH NO SMOKE OR ARCING,2020-03-03 02:14:34
169,3/3/2020 2:20:17 AM,2412 DENNYWOOD DR,36.178459,-86.681069,TRANSFORMER OUTSIDE WIRE OR POLE,2020-03-03 02:20:17
172,3/3/2020 2:22:59 AM,4496 CLEECES FERRY RD,36.173577,-86.932479,WIRES DOWN WITH NO SMOKE OR ARCING,2020-03-03 02:22:59
196,3/3/2020 2:56:58 AM,1731 24TH AV N,36.178716,-86.815840,WIRES DOWN WITH NO SMOKE OR ARCING,2020-03-03 02:56:58
208,3/3/2020 3:12:58 AM,1056 MALLOW DR,36.257768,-86.679547,WIRES DOWN WITH NO SMOKE OR ARCING,2020-03-03 03:12:58


In [23]:
tornado_wires.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 138 to 255
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   IncidentDate              12 non-null     object        
 1   Location                  12 non-null     object        
 2   Latitude1                 12 non-null     float64       
 3   Longitude1                12 non-null     float64       
 4   IncidentTypeDescription1  12 non-null     object        
 5   Timestamp                 12 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 672.0+ bytes


In [24]:
tornado_odor = late_tornado_comp[late_tornado_comp['IncidentTypeDescription1'].str.contains('ODOR')]

In [25]:
tornado_odor

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1,Timestamp
131,3/3/2020 2:02:32 AM,S 9TH ST / MAIN ST,36.175964,-86.757246,OUTSIDE NATURAL GAS ODOR COMMERCIAL LINE TANK ...,2020-03-03 02:02:32
132,3/3/2020 2:03:06 AM,111 EVANDER ST,36.175976,-86.740844,OUTSIDE ODOR,2020-03-03 02:03:06
133,3/3/2020 2:04:00 AM,938 W EASTLAND AV,36.184138,-86.754363,OUTSIDE ODOR,2020-03-03 02:04:00
134,3/3/2020 2:04:00 AM,938 W EASTLAND AV,36.184138,-86.754363,OUTSIDE ODOR,2020-03-03 02:04:00
146,3/3/2020 2:07:30 AM,2808 HUNTLEIGH DR,36.172935,-86.725732,HOUSE NATURAL GAS ODOR,2020-03-03 02:07:30
148,3/3/2020 2:08:54 AM,3213 FRENO LN,36.178065,-86.677046,OUTSIDE ODOR,2020-03-03 02:08:54
158,3/3/2020 2:11:37 AM,2640 DELK AV,36.180448,-86.821786,OUTSIDE ODOR,2020-03-03 02:11:37
159,3/3/2020 2:13:00 AM,310 N 9TH ST,36.180008,-86.758802,OUTSIDE ODOR,2020-03-03 02:13:00
163,3/3/2020 2:15:55 AM,1000 JEFFERSON ST,36.172742,-86.795228,OUTSIDE ODOR,2020-03-03 02:15:55
167,3/3/2020 2:19:06 AM,3500 JOHN A MERRITT BLVD,36.166803,-86.829485,OUTSIDE ODOR,2020-03-03 02:19:06


In [26]:
tornado_odor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 131 to 227
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   IncidentDate              22 non-null     object        
 1   Location                  22 non-null     object        
 2   Latitude1                 22 non-null     float64       
 3   Longitude1                22 non-null     float64       
 4   IncidentTypeDescription1  22 non-null     object        
 5   Timestamp                 22 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 1.2+ KB
